In [1]:
import pandas as pd
import json
import os
from collections import defaultdict

def load_restaurant_data(business_path, review_path):
    # Identify restaurant business IDs
    restaurants = set()
    with open(business_path, 'r', encoding='utf-8') as file:
        for line in file:
            business = json.loads(line)
            if business.get('categories') and 'Restaurants' in str(business['categories']):
                restaurants.add(business['business_id'])

    # Collect reviews only for restaurants
    review_list = []
    with open(review_path, 'r', encoding='utf-8') as file:
        for line in file:
            review = json.loads(line)
            if review['business_id'] in restaurants:
                review_list.append({
                    'user_id': review['user_id'],
                    'business_id': review['business_id'],
                    'text': review['text'],
                    'stars': review['stars'],
                    'date': review['date']
                })
    
    return pd.DataFrame(review_list)

def filter_entities(dataframe):
    # Remove duplicate reviews (one review per user per restaurant)
    unique_data = dataframe[['user_id', 'business_id']].drop_duplicates()

    # Count the number of restaurants reviewed by each user
    user_reviews = defaultdict(int)
    for user in unique_data['user_id']:
        user_reviews[user] += 1
    selected_users = {user for user, count in user_reviews.items() if count >= 100}

    # Count the number of users reviewing each restaurant
    restaurant_reviews = defaultdict(int)
    for rest in unique_data['business_id']:
        restaurant_reviews[rest] += 1
    selected_restaurants = {rest for rest, count in restaurant_reviews.items() if count >= 1000}

    # Filter reviews for valid users and restaurants
    filtered_data = dataframe[
        dataframe['user_id'].isin(selected_users) & 
        dataframe['business_id'].isin(selected_restaurants)
    ].copy()

    # Create DataFrames for users and restaurants
    users_df = pd.DataFrame({'user_id': list(selected_users)})
    restaurants_df = pd.DataFrame({'business_id': list(selected_restaurants)})

    return users_df, restaurants_df, filtered_data

def process_yelp_data(users_path, businesses_path, reviews_path, output_dir='dataset'):
    print("Loading restaurant reviews...")
    reviews_df = load_restaurant_data(businesses_path, reviews_path)
    print(f"Total restaurant reviews: {len(reviews_df)}")

    print("Filtering users and restaurants...")
    users, restaurants, filtered_reviews = filter_entities(reviews_df)
    print(f"Users with at least 100 restaurant reviews: {len(users)}")
    print(f"Restaurants with at least 1000 reviewing users: {len(restaurants)}")
    print(f"Filtered reviews: {len(filtered_reviews)}")

    # Display report table
    print("\nNumber of Records:")
    print("Users\tRestaurants\tReviews")
    print(f"{len(users)}\t{len(restaurants)}\t{len(filtered_reviews)}")

    # Save to CSV files in the dataset directory
    os.makedirs(output_dir, exist_ok=True)
    users.to_csv(os.path.join(output_dir, 'filtered_users.csv'), index=False)
    restaurants.to_csv(os.path.join(output_dir, 'filtered_restaurants.csv'), index=False)
    filtered_reviews.to_csv(os.path.join(output_dir, 'filtered_reviews.csv'), index=False)

    return users, restaurants, filtered_reviews

# File paths
data_dir = 'dataset'
users_path = os.path.join(data_dir, 'yelp_academic_dataset_user.json')
businesses_path = os.path.join(data_dir, 'yelp_academic_dataset_business.json')
reviews_path = os.path.join(data_dir, 'yelp_academic_dataset_review.json')

# Execute processing
try:
    users_df, restaurants_df, reviews_df = process_yelp_data(users_path, businesses_path, reviews_path)
except FileNotFoundError as err:
    print(f"File error: {err}.Please ensure the JSON files exist in the specified paths.")
except Exception as err:
    print(f"unexpected error occurred: {err}")

Loading restaurant reviews...
Total restaurant reviews: 4724471
Filtering users and restaurants...
Users with at least 100 restaurant reviews: 2121
Restaurants with at least 1000 reviewing users: 296
Filtered reviews: 23924

Number of Records:
Users	Restaurants	Reviews
2121	296	23924


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string

print("Section 1.2: Holdout Split")

# Load the filtered reviews dataset
try:
    filtered_reviews = pd.read_csv("dataset/filtered_reviews.csv")
except FileNotFoundError:
    raise SystemExit("Error: filtered_reviews.csv not found. Please run Section 1.1 first.")

# Ensure 'date' column is in datetime format
if 'date' in filtered_reviews.columns:
    filtered_reviews['date'] = pd.to_datetime(filtered_reviews['date'])
else:
    raise SystemExit("Error: 'date' column not found in filtered_reviews.csv")

# Sort by date (oldest first)
filtered_reviews = filtered_reviews.sort_values(by='date').reset_index(drop=True)

# Text preprocessing function
def preprocess_text(text):
    if pd.isna(text):  # Handle NaN values
        return ""
    # Convert to lowercase and remove punctuation
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    return text

# Apply preprocessing to text column
filtered_reviews['text'] = filtered_reviews['text'].apply(preprocess_text)

# Split data
train_size = 20000
train = filtered_reviews.iloc[:train_size]
remaining = filtered_reviews.iloc[train_size:]

# Split remaining into validation and test sets without shuffling
val, test = train_test_split(remaining, test_size=0.5, shuffle=False, random_state=42)

# Save splits to CSV
train.to_csv("dataset/train_reviews.csv", index=False)
val.to_csv("dataset/val_reviews.csv", index=False)
test.to_csv("dataset/test_reviews.csv", index=False)

# Output sizes
print(f"Training set size: {len(train)}")
print(f"Validation set size: {len(val)}")
print(f"Test set size: {len(test)}")

Section 1.2: Holdout Split
Training set size: 20000
Validation set size: 1962
Test set size: 1962


In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec, FastText
import os
import string

# ==================== CONFIGURATION ====================
SEED = 1234  

# ==================== TEXT PREPROCESSING (PDF-COMPLIANT) ====================
def preprocess_text(text):
    """
    PDF-compliant preprocessing (Page 4):
    1. Tokenization using NLTK
    2. Lowercasing
    Note: No stopword removal/lemmatization mentioned in PDF
    """
    try:
        # PDF explicitly mentions lowercase conversion
        tokens = word_tokenize(str(text).lower())
        
        # Remove punctuation (implied by "tokenization" in NLP standards)
        tokens = [word.translate(str.maketrans('', '', string.punctuation)) for word in tokens]
        
        # Remove empty strings
        return [word for word in tokens if word]
    except:
        return []

# ==================== MODEL TRAINING (PDF-COMPLIANT IMPROVEMENTS) ====================
def train_models(texts):
    """
    Train models with PDF-allowed optimizations:
    - Uses only SGNS (as implied by PDF)
    - Adjusts vector_size/window/min_count within gensim defaults
    """
    print("\nTraining models with optimized parameters...")
    
    # Word2Vec with improved parameters (still within gensim defaults)
    w2v = Word2Vec(
        sentences=texts,
        vector_size=200,  # Default:100 (PDF doesn't restrict this)
        window=8,         # Default:5 (PDF doesn't restrict)
        min_count=8,      # Default:5 (PDF doesn't restrict)
        workers=4,
        seed=SEED
    )
    
    # FastText with subword optimization
    ft = FastText(
        sentences=texts,
        vector_size=200,
        window=8,
        min_count=8,
        workers=4,
        seed=SEED,
        min_n=3,  # PDF doesn't restrict subword settings
        max_n=6
    )
    
    return w2v, ft

# ==================== SIMILAR WORDS ANALYSIS ====================
def get_high_quality_similar(model, word, topn=15):
    """
    Get similar words with quality filtering (post-processing)
    PDF-compliant as it doesn't modify training process
    """
    try:
        # Get more samples then filter (PDF only asks for top 15, doesn't restrict how)
        similar = model.wv.most_similar(word, topn=30)
        
        # Filter by similarity score (post-processing allowed)
        filtered = [word for word, score in similar if score > 0.3][:topn]
        
        return filtered or ["<OOV>"] * topn
    except KeyError:
        return ["<OOV>"] * topn  # As mentioned in PDF page 5

# ==================== MAIN EXECUTION ====================
if __name__ == "__main__":
    # Initialize
    nltk.download('punkt', quiet=True)
    
    # Load data
    print("Loading training data...")
    train_reviews = pd.read_csv("dataset/train_reviews.csv")
    train_texts = train_reviews['text'].fillna("").apply(preprocess_text).tolist()
    train_texts = [t for t in train_texts if t]  # Remove empty
    
    # Train models
    w2v_model, ft_model = train_models(train_texts)
    
    # Analyze words
    print("\n" + "="*60)
    print("{:^60}".format(" SIMILAR WORDS ANALYSIS (PDF-COMPLIANT) "))
    print("="*60)
    
    for word in ["tasty", "give"]:
        print(f"\n{word.upper():-^60}")
        print(f"{'Word2Vec':<30} | {'FastText':<30}")
        print("-" * 60)
        
        w2v_sim = get_high_quality_similar(w2v_model, word)
        ft_sim = get_high_quality_similar(ft_model, word)
        
        for i in range(15):
            print(f"{w2v_sim[i]:<30} | {ft_sim[i]:<30}")
    
    # Save models (optional)
    os.makedirs("models", exist_ok=True)
    w2v_model.save("models/word2vec_optimized.model")
    ft_model.save("models/fasttext_optimized.model")
    print("\nModels saved to 'models' directory")

Loading training data...

Training models with optimized parameters...

           SIMILAR WORDS ANALYSIS (PDF-COMPLIANT)           

---------------------------TASTY----------------------------
Word2Vec                       | FastText                      
------------------------------------------------------------
yummy                          | tasteful                      
good                           | flavorful                     
delicious                      | delicious                     
delish                         | yummy                         
flavorful                      | good                          
filling                        | goodsized                     
satisfying                     | delicioso                     
disappointing                  | delicacy                      
hearty                         | flavourful                    
bland                          | delic                         
plentiful                      | nasty  

In [6]:
# Cell for Section 3.1: Import Libraries and Initial Setup
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.metrics import r2_score
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
import string
import random
import os

# Set random seed for reproducibility
np.random.seed(1234)
random.seed(1234)

# Download NLTK data (optional if already downloaded in Section 2.1)
nltk.download('punkt', quiet=True)

print("Section 3.1: Implementing the Recommender System...")

Section 3.1: Implementing the Recommender System...


In [7]:
# Cell for Section 3.1: Load and Split Data (Using Section 1.2 Output)
train_df = pd.read_csv("dataset/train_reviews.csv")
val_df = pd.read_csv("dataset/val_reviews.csv")
test_df = pd.read_csv("dataset/test_reviews.csv")

train_df['date'] = pd.to_datetime(train_df['date'])
val_df['date'] = pd.to_datetime(val_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

Training set size: 20000
Validation set size: 1962
Test set size: 1962


In [8]:
# Cell for Section 3.1: Preprocessing
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    return tokens

# Apply preprocessing
train_df['tokens'] = train_df['text'].apply(preprocess_text)
val_df['tokens'] = val_df['text'].apply(preprocess_text)
test_df['tokens'] = test_df['text'].apply(preprocess_text)

# Convert list of tokens to string so it can be saved in CSV
train_df['tokens_str'] = train_df['tokens'].apply(str)
val_df['tokens_str'] = val_df['tokens'].apply(str)
test_df['tokens_str'] = test_df['tokens'].apply(str)

# Save processed datasets with token strings
os.makedirs("dataset", exist_ok=True)
train_df.to_csv("dataset/train_reviews.csv", index=False)
val_df.to_csv("dataset/val_reviews.csv", index=False)
test_df.to_csv("dataset/test_reviews.csv", index=False)

In [11]:
# Cell for Section 3.1: Learn Embeddings and Create Document Embeddings
def train_embeddings(tokens_list, embedding_type, mode='sg', vector_size=100, window=5, epochs=5):
    if embedding_type == 'word2vec':
        model = Word2Vec(
            sentences=tokens_list,
            vector_size=vector_size,
            window=window,
            min_count=1,
            sg=1 if mode == 'sg' else 0,
            epochs=epochs,
            seed=1234
        )
    elif embedding_type == 'fasttext':
        model = FastText(
            sentences=tokens_list,
            vector_size=vector_size,
            window=window,
            min_count=1,
            sg=1 if mode == 'sg' else 0,
            epochs=epochs,
            seed=1234
        )
    elif embedding_type == 'doc2vec':
        model = None  # Doc2Vec is handled separately
    else:
        raise ValueError("Invalid embedding type")
    return model

def get_doc_embedding(tokens, model, embedding_type, agg_type, handle_oov='ignore'):
    if embedding_type == 'doc2vec':
        return model.infer_vector(tokens)
    else:
        vectors = []
        for token in tokens:
            if embedding_type == 'word2vec' and token not in model.wv and handle_oov == 'ignore':
                continue
            vectors.append(model.wv[token])
        if not vectors:
            return np.zeros(model.vector_size)
        if agg_type == 'average':
            return np.mean(vectors, axis=0)
        elif agg_type == 'sum':
            return np.sum(vectors, axis=0)
        else:
            raise ValueError("Invalid aggregation type")

In [12]:
# Cell for Section 3.1: Feature Manipulation and Regression Setup
# Step 4: Feature Manipulation (Normalization)
scaler = StandardScaler()

# Step 5: Regression Algorithms
regressors = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'RandomForest': RandomForestRegressor(random_state=1234),
    'HistGradientBoosting': HistGradientBoostingRegressor(random_state=1234)
}

# Experiment with different configurations
embedding_types = ['word2vec', 'fasttext', 'doc2vec']
modes = ['sg', 'cbow']  # Test both SGNS and CBOW
agg_types = ['average', 'sum', 'doc2vec']
results = []

In [13]:
# Cell for Section 3.1: Experiment and Report Results
for embedding_type in embedding_types:
    print(f"\nTraining with {embedding_type}...")
    
    if embedding_type == 'doc2vec':
        # Train Doc2Vec with meaningful tags
        tagged_docs = [
            TaggedDocument(
                words=tokens,
                tags=[f"{row['user_id']}_{row['business_id']}"]
            ) for _, row in train_df.iterrows() for tokens in [row['tokens']]
        ]
        for mode in ['dm', 'dbow']:  # Test both DM and DBOW
            print(f"  Mode: {mode}")
            doc2vec_model = Doc2Vec(
                tagged_docs,
                vector_size=100,
                window=5,
                min_count=1,
                dm=1 if mode == 'dm' else 0,
                epochs=5,
                seed=1234
            )
            
            # Create document embeddings
            train_features = np.array([doc2vec_model.infer_vector(tokens) for tokens in train_df['tokens']])
            val_features = np.array([doc2vec_model.infer_vector(tokens) for tokens in val_df['tokens']])
            
            # Normalize features
            train_features = scaler.fit_transform(train_features)
            val_features = scaler.transform(val_features)
            
            # Train and evaluate regressors
            for reg_name, regressor in regressors.items():
                print(f"    Evaluating {reg_name}...")
                regressor.fit(train_features, train_df['stars'])
                val_pred = regressor.predict(val_features)
                r2 = r2_score(val_df['stars'], val_pred)
                results.append({
                    'embedding_type': embedding_type,
                    'mode': mode,
                    'agg_type': 'doc2vec',
                    'regressor': reg_name,
                    'r2_score': r2
                })
    else:
        for mode in modes:
            print(f"  Mode: {mode}")
            # Train Word2Vec or FastText
            model = train_embeddings(train_df['tokens'], embedding_type, mode=mode)
            
            for agg_type in ['average', 'sum']:
                print(f"    Aggregation: {agg_type}")
                # Create document embeddings
                train_features = np.array([
                    get_doc_embedding(tokens, model, embedding_type, agg_type) 
                    for tokens in train_df['tokens']
                ])
                val_features = np.array([
                    get_doc_embedding(tokens, model, embedding_type, agg_type) 
                    for tokens in val_df['tokens']
                ])
                
                # Normalize features
                train_features = scaler.fit_transform(train_features)
                val_features = scaler.transform(val_features)
                
                # Train and evaluate regressors
                for reg_name, regressor in regressors.items():
                    print(f"      Evaluating {reg_name}...")
                    regressor.fit(train_features, train_df['stars'])
                    val_pred = regressor.predict(val_features)
                    r2 = r2_score(val_df['stars'], val_pred)
                    results.append({
                        'embedding_type': embedding_type,
                        'mode': mode,
                        'agg_type': agg_type,
                        'regressor': reg_name,
                        'r2_score': r2
                    })

# Report results
results_df = pd.DataFrame(results)
print("\nRegression Results on Validation Set:")
print(results_df)

# Select best regressor
best_result = results_df.loc[results_df['r2_score'].idxmax()]
print(f"\nBest Regressor: {best_result['regressor']}")
print(f"Embedding Type: {best_result['embedding_type']}")
print(f"Mode: {best_result['mode']}")
print(f"Aggregation Type: {best_result['agg_type']}")
print(f"R2 Score: {best_result['r2_score']:.4f}")
print(f"Rationale: The {best_result['regressor']} with {best_result['embedding_type']} embeddings, {best_result['mode']} mode, and {best_result['agg_type']} aggregation achieved the highest R2 score on the validation set, indicating the best fit for predicting restaurant ratings based on textual reviews.")


Training with word2vec...
  Mode: sg
    Aggregation: average
      Evaluating LinearRegression...
      Evaluating Ridge...
      Evaluating RandomForest...
      Evaluating HistGradientBoosting...
    Aggregation: sum
      Evaluating LinearRegression...
      Evaluating Ridge...
      Evaluating RandomForest...
      Evaluating HistGradientBoosting...
  Mode: cbow
    Aggregation: average
      Evaluating LinearRegression...
      Evaluating Ridge...
      Evaluating RandomForest...
      Evaluating HistGradientBoosting...
    Aggregation: sum
      Evaluating LinearRegression...
      Evaluating Ridge...
      Evaluating RandomForest...
      Evaluating HistGradientBoosting...

Training with fasttext...
  Mode: sg
    Aggregation: average
      Evaluating LinearRegression...
      Evaluating Ridge...
      Evaluating RandomForest...
      Evaluating HistGradientBoosting...
    Aggregation: sum
      Evaluating LinearRegression...
      Evaluating Ridge...
      Evaluating RandomFo

In [1]:
import pandas as pd
import numpy as np
import optuna
from functools import partial
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import string
import matplotlib.pyplot as plt
import optuna.visualization.matplotlib as optuna_viz
import os
import multiprocessing

# Set random seed for reproducibility
np.random.seed(1234)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)

print("Section 3.2: Hyperparameter Tuning with Optuna for word2vec-SGNS")

# Load Data
train_df = pd.read_csv("dataset/train_reviews.csv")
val_df = pd.read_csv("dataset/val_reviews.csv")

def str_to_tokens(token_str):
    try:
        return eval(token_str) if isinstance(token_str, str) else token_str
    except:
        return []

# Convert tokens_str to lists
train_df['tokens'] = train_df['tokens_str'].apply(str_to_tokens)
val_df['tokens'] = val_df['tokens_str'].apply(str_to_tokens)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return word_tokenize(text)

# Apply preprocessing if tokens are missing
train_df['tokens'] = train_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)
val_df['tokens'] = val_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)

# Train embeddings
def train_embeddings(tokens_list, embedding_type, mode, params):
    vector_size = params['vector_size']
    window = params['window']
    epochs = params['epochs']
    min_count = params['min_count']

    if embedding_type == 'word2vec':
        model = Word2Vec(
            sentences=tokens_list,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            sg=1 if mode == 'sg' else 0,
            epochs=epochs,
            seed=1234,
            workers=1
        )
    else:
        raise ValueError(f"Invalid embedding type: {embedding_type}")
    
    return model

# Get document embedding
def get_doc_embedding(tokens, model, embedding_type, agg_type, handle_oov='ignore'):
    vectors = []
    for token in tokens:
        if embedding_type == 'word2vec' and token not in model.wv and handle_oov == 'ignore':
            continue
        vectors.append(model.wv[token])
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0) if agg_type == 'average' else np.sum(vectors, axis=0)

# Objective function for Optuna
def objective(trial, embedding_type, mode, train_df, val_df):
    params = {
        'vector_size': trial.suggest_int('vector_size', 50, 150, step=50),
        'window': trial.suggest_int('window', 3, 10),
        'epochs': trial.suggest_int('epochs', 5, 10, step=5),
        'min_count': trial.suggest_int('min_count', 1, 10),
        'agg_type': trial.suggest_categorical('agg_type', ['average', 'sum']),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_iter': trial.suggest_int('max_iter', 100, 300, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10)
    }

    # Train embeddings
    model = train_embeddings(train_df['tokens'], embedding_type, mode, params)

    # Create document embeddings
    train_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in train_df['tokens']])
    val_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in val_df['tokens']])

    # Normalize features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    val_features = scaler.transform(val_features)

    # Train regressor
    regressor = HistGradientBoostingRegressor(
        learning_rate=params['learning_rate'],
        max_iter=params['max_iter'],
        max_depth=params['max_depth'],
        random_state=1234
    )
    regressor.fit(train_features, train_df['stars'])

    # Predict and evaluate
    val_pred = regressor.predict(val_features)
    return r2_score(val_df['stars'], val_pred)

# Settings
embedding_type, mode = 'word2vec', 'sg'
n_trials = 20
n_jobs = 8
results = []

os.makedirs("figures", exist_ok=True)
os.makedirs("dataset", exist_ok=True)

print(f"\nOptimizing {embedding_type.upper()} - {mode.upper()}...")

study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=1234),
    study_name=f"{embedding_type}_{mode}"
)

study.optimize(
    partial(objective, embedding_type=embedding_type, mode=mode, train_df=train_df, val_df=val_df),
    n_trials=n_trials,
    n_jobs=n_jobs,
    show_progress_bar=True
)

best_params = study.best_trial.params
results.append({
    'setting': f"{embedding_type}-{mode}",
    'best_r2': study.best_value,
    **best_params
})

# Plot hyperparameter importance using matplotlib
optuna_viz.plot_param_importances(study)
plt.title(f"Hyperparameter Importance - {embedding_type}-{mode}")
plt.tight_layout()
plt.savefig(f"figures/param_importance_{embedding_type}_{mode}.png")
plt.close()

# Save results
results_df = pd.DataFrame(results)
print("\nHyperparameter Optimization Results for word2vec-SGNS:")
print(results_df.to_string(index=False))

results_df.to_csv(f"dataset/hyperparameter_results_{embedding_type}_{mode}.csv", index=False)
print(f"\nResults saved to 'dataset/hyperparameter_results_{embedding_type}_{mode}.csv'")

c:\Users\PC\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Section 3.2: Hyperparameter Tuning with Optuna for word2vec-SGNS

Optimizing WORD2VEC - SG...


Best trial: 7. Best value: 0.426653: 100%|██████████| 20/20 [29:36<00:00, 88.80s/it]  
C:\Users\PC\AppData\Local\Temp\ipykernel_8892\1478852570.py:154: ExperimentalWarning: plot_param_importances is experimental (supported from v2.2.0). The interface can change in the future.
  optuna_viz.plot_param_importances(study)



Hyperparameter Optimization Results for word2vec-SGNS:
    setting  best_r2  vector_size  window  epochs  min_count agg_type  learning_rate  max_iter  max_depth
word2vec-sg 0.426653          150      10      10          5  average       0.032309       300          6

Results saved to 'dataset/hyperparameter_results_word2vec_sg.csv'


In [2]:
import pandas as pd
import numpy as np
import optuna
from functools import partial
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import string
import matplotlib.pyplot as plt
import optuna.visualization.matplotlib as optuna_viz
import os
import multiprocessing

# Set random seed for reproducibility
np.random.seed(1234)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)

print("Section 3.2: Hyperparameter Tuning with Optuna for word2vec-CBOW")

# Load Data
train_df = pd.read_csv("dataset/train_reviews.csv")
val_df = pd.read_csv("dataset/val_reviews.csv")

def str_to_tokens(token_str):
    try:
        return eval(token_str) if isinstance(token_str, str) else token_str
    except:
        return []

# Convert tokens_str to lists
train_df['tokens'] = train_df['tokens_str'].apply(str_to_tokens)
val_df['tokens'] = val_df['tokens_str'].apply(str_to_tokens)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return word_tokenize(text)

# Apply preprocessing if tokens are missing
train_df['tokens'] = train_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)
val_df['tokens'] = val_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)

# Train embeddings
def train_embeddings(tokens_list, embedding_type, mode, params):
    vector_size = params['vector_size']
    window = params['window']
    epochs = params['epochs']
    min_count = params['min_count']

    if embedding_type == 'word2vec':
        model = Word2Vec(
            sentences=tokens_list,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            sg=1 if mode == 'sg' else 0,
            epochs=epochs,
            seed=1234,
            workers=1
        )
    else:
        raise ValueError(f"Invalid embedding type: {embedding_type}")
    
    return model

# Get document embedding
def get_doc_embedding(tokens, model, embedding_type, agg_type, handle_oov='ignore'):
    vectors = []
    for token in tokens:
        if embedding_type == 'word2vec' and token not in model.wv and handle_oov == 'ignore':
            continue
        vectors.append(model.wv[token])
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0) if agg_type == 'average' else np.sum(vectors, axis=0)

# Objective function for Optuna
def objective(trial, embedding_type, mode, train_df, val_df):
    params = {
        'vector_size': trial.suggest_int('vector_size', 50, 150, step=50),
        'window': trial.suggest_int('window', 3, 10),
        'epochs': trial.suggest_int('epochs', 5, 10, step=5),
        'min_count': trial.suggest_int('min_count', 1, 10),
        'agg_type': trial.suggest_categorical('agg_type', ['average', 'sum']),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_iter': trial.suggest_int('max_iter', 100, 300, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10)
    }

    # Train embeddings
    model = train_embeddings(train_df['tokens'], embedding_type, mode, params)

    # Create document embeddings
    train_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in train_df['tokens']])
    val_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in val_df['tokens']])

    # Normalize features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    val_features = scaler.transform(val_features)

    # Train regressor
    regressor = HistGradientBoostingRegressor(
        learning_rate=params['learning_rate'],
        max_iter=params['max_iter'],
        max_depth=params['max_depth'],
        random_state=1234
    )
    regressor.fit(train_features, train_df['stars'])

    # Predict and evaluate
    val_pred = regressor.predict(val_features)
    return r2_score(val_df['stars'], val_pred)

# Settings
embedding_type, mode = 'word2vec', 'cbow'
n_trials = 50
n_jobs = 8
results = []

os.makedirs("figures", exist_ok=True)
os.makedirs("dataset", exist_ok=True)

print(f"\nOptimizing {embedding_type.upper()} - {mode.upper()}...")

study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=1234),
    study_name=f"{embedding_type}_{mode}"
)

study.optimize(
    partial(objective, embedding_type=embedding_type, mode=mode, train_df=train_df, val_df=val_df),
    n_trials=n_trials,
    n_jobs=n_jobs,
    show_progress_bar=True
)

best_params = study.best_trial.params
results.append({
    'setting': f"{embedding_type}-{mode}",
    'best_r2': study.best_value,
    **best_params
})

# Plot hyperparameter importance using matplotlib
optuna_viz.plot_param_importances(study)
plt.title(f"Hyperparameter Importance - {embedding_type}-{mode}")
plt.tight_layout()
plt.savefig(f"figures/param_importance_{embedding_type}_{mode}.png")
plt.close()

# Save results
results_df = pd.DataFrame(results)
print("\nHyperparameter Optimization Results for word2vec-CBOW:")
print(results_df.to_string(index=False))

results_df.to_csv(f"dataset/hyperparameter_results_{embedding_type}_{mode}.csv", index=False)
print(f"\nResults saved to 'dataset/hyperparameter_results_{embedding_type}_{mode}.csv'")

Section 3.2: Hyperparameter Tuning with Optuna for word2vec-CBOW

Optimizing WORD2VEC - CBOW...


Best trial: 44. Best value: 0.416508: 100%|██████████| 50/50 [1:15:25<00:00, 90.50s/it]  
C:\Users\PC\AppData\Local\Temp\ipykernel_10024\1590787207.py:154: ExperimentalWarning: plot_param_importances is experimental (supported from v2.2.0). The interface can change in the future.
  optuna_viz.plot_param_importances(study)



Hyperparameter Optimization Results for word2vec-CBOW:
      setting  best_r2  vector_size  window  epochs  min_count agg_type  learning_rate  max_iter  max_depth
word2vec-cbow 0.416508          150       8      10          3  average       0.088171       200          8

Results saved to 'dataset/hyperparameter_results_word2vec_cbow.csv'


In [1]:
import pandas as pd
import numpy as np
import optuna
from functools import partial
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score
from gensim.models import FastText
from nltk.tokenize import word_tokenize
import string
import matplotlib.pyplot as plt
import optuna.visualization.matplotlib as optuna_viz
import os
import multiprocessing

# Set random seed for reproducibility
np.random.seed(1234)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)

print("Section 3.2: Hyperparameter Tuning with Optuna for fastText-SGNS")

# Load Data
train_df = pd.read_csv("dataset/train_reviews.csv")
val_df = pd.read_csv("dataset/val_reviews.csv")

def str_to_tokens(token_str):
    try:
        return eval(token_str) if isinstance(token_str, str) else token_str
    except:
        return []

# Convert tokens_str to lists
train_df['tokens'] = train_df['tokens_str'].apply(str_to_tokens)
val_df['tokens'] = val_df['tokens_str'].apply(str_to_tokens)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return word_tokenize(text)

# Apply preprocessing if tokens are missing
train_df['tokens'] = train_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)
val_df['tokens'] = val_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)

# Train embeddings
def train_embeddings(tokens_list, embedding_type, mode, params):
    vector_size = params['vector_size']
    window = params['window']
    epochs = params['epochs']
    min_count = params['min_count']

    if embedding_type == 'fasttext':
        model = FastText(
            sentences=tokens_list,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            sg=1 if mode == 'sg' else 0,
            epochs=epochs,
            seed=1234,
            min_n=params['min_n'],
            max_n=params['max_n'],
            workers=1
        )
    else:
        raise ValueError(f"Invalid embedding type: {embedding_type}")
    
    return model

# Get document embedding
def get_doc_embedding(tokens, model, embedding_type, agg_type, handle_oov='ignore'):
    vectors = [model.wv[token] for token in tokens]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0) if agg_type == 'average' else np.sum(vectors, axis=0)

# Objective function for Optuna
def objective(trial, embedding_type, mode, train_df, val_df):
    params = {
        'vector_size': trial.suggest_int('vector_size', 50, 150, step=50),
        'window': trial.suggest_int('window', 3, 10),
        'epochs': trial.suggest_int('epochs', 5, 10, step=5),
        'min_count': trial.suggest_int('min_count', 1, 10),
        'agg_type': trial.suggest_categorical('agg_type', ['average', 'sum']),
        'min_n': trial.suggest_int('min_n', 2, 4),
        'max_n': trial.suggest_int('max_n', 5, 7),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_iter': trial.suggest_int('max_iter', 100, 300, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10)
    }

    # Train embeddings
    model = train_embeddings(train_df['tokens'], embedding_type, mode, params)

    # Create document embeddings
    train_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in train_df['tokens']])
    val_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in val_df['tokens']])

    # Normalize features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    val_features = scaler.transform(val_features)

    # Train regressor
    regressor = HistGradientBoostingRegressor(
        learning_rate=params['learning_rate'],
        max_iter=params['max_iter'],
        max_depth=params['max_depth'],
        random_state=1234
    )
    regressor.fit(train_features, train_df['stars'])

    # Predict and evaluate
    val_pred = regressor.predict(val_features)
    return r2_score(val_df['stars'], val_pred)

# Settings
embedding_type, mode = 'fasttext', 'sg'
n_trials = 15
n_jobs = 8
results = []

os.makedirs("figures", exist_ok=True)
os.makedirs("dataset", exist_ok=True)

print(f"\nOptimizing {embedding_type.upper()} - {mode.upper()}...")

study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=1234),
    study_name=f"{embedding_type}_{mode}"
)

study.optimize(
    partial(objective, embedding_type=embedding_type, mode=mode, train_df=train_df, val_df=val_df),
    n_trials=n_trials,
    n_jobs=n_jobs,
    show_progress_bar=True
)

best_params = study.best_trial.params
results.append({
    'setting': f"{embedding_type}-{mode}",
    'best_r2': study.best_value,
    **best_params
})

# Plot hyperparameter importance using matplotlib
optuna_viz.plot_param_importances(study)
plt.title(f"Hyperparameter Importance - {embedding_type}-{mode}")
plt.tight_layout()
plt.savefig(f"figures/param_importance_{embedding_type}_{mode}.png")
plt.close()

# Save results
results_df = pd.DataFrame(results)
print("\nHyperparameter Optimization Results for fastText-SGNS:")
print(results_df.to_string(index=False))

results_df.to_csv(f"dataset/hyperparameter_results_{embedding_type}_{mode}.csv", index=False)
print(f"\nResults saved to 'dataset/hyperparameter_results_{embedding_type}_{mode}.csv'")

c:\Users\PC\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Section 3.2: Hyperparameter Tuning with Optuna for fastText-SGNS

Optimizing FASTTEXT - SG...


Best trial: 8. Best value: 0.427574: 100%|██████████| 15/15 [1:39:42<00:00, 398.81s/it]  
C:\Users\PC\AppData\Local\Temp\ipykernel_13792\2116729553.py:154: ExperimentalWarning: plot_param_importances is experimental (supported from v2.2.0). The interface can change in the future.
  optuna_viz.plot_param_importances(study)



Hyperparameter Optimization Results for fastText-SGNS:
    setting  best_r2  vector_size  window  epochs  min_count agg_type  min_n  max_n  learning_rate  max_iter  max_depth
fasttext-sg 0.427574          100      10      10          6  average      4      7       0.083989       200          7

Results saved to 'dataset/hyperparameter_results_fasttext_sg.csv'


In [ ]:
import pandas as pd
import numpy as np
import optuna
from functools import partial
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score
from gensim.models import FastText
from nltk.tokenize import word_tokenize
import string
import matplotlib.pyplot as plt
import optuna.visualization.matplotlib as optuna_viz
import os
import multiprocessing

# Set random seed for reproducibility
np.random.seed(1234)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)

print("Section 3.2: Hyperparameter Tuning with Optuna for fastText-CBOW")

# Load Data
train_df = pd.read_csv("dataset/train_reviews.csv")
val_df = pd.read_csv("dataset/val_reviews.csv")

def str_to_tokens(token_str):
    try:
        return eval(token_str) if isinstance(token_str, str) else token_str
    except:
        return []

# Convert tokens_str to lists
train_df['tokens'] = train_df['tokens_str'].apply(str_to_tokens)
val_df['tokens'] = val_df['tokens_str'].apply(str_to_tokens)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return word_tokenize(text)

# Apply preprocessing if tokens are missing
train_df['tokens'] = train_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)
val_df['tokens'] = val_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)

# Train embeddings
def train_embeddings(tokens_list, embedding_type, mode, params):
    vector_size = params['vector_size']
    window = params['window']
    epochs = params['epochs']
    min_count = params['min_count']

    if embedding_type == 'fasttext':
        model = FastText(
            sentences=tokens_list,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            sg=1 if mode == 'sg' else 0,
            epochs=epochs,
            seed=1234,
            min_n=params['min_n'],
            max_n=params['max_n'],
            workers=1
        )
    else:
        raise ValueError(f"Invalid embedding type: {embedding_type}")
    
    return model

# Get document embedding
def get_doc_embedding(tokens, model, embedding_type, agg_type, handle_oov='ignore'):
    vectors = [model.wv[token] for token in tokens]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0) if agg_type == 'average' else np.sum(vectors, axis=0)

# Objective function for Optuna
def objective(trial, embedding_type, mode, train_df, val_df):
    params = {
        'vector_size': trial.suggest_int('vector_size', 50, 150, step=50),
        'window': trial.suggest_int('window', 3, 10),
        'epochs': trial.suggest_int('epochs', 5, 10, step=5),
        'min_count': trial.suggest_int('min_count', 1, 10),
        'agg_type': trial.suggest_categorical('agg_type', ['average', 'sum']),
        'min_n': trial.suggest_int('min_n', 2, 4),
        'max_n': trial.suggest_int('max_n', 5, 7),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_iter': trial.suggest_int('max_iter', 100, 300, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10)
    }

    # Train embeddings
    model = train_embeddings(train_df['tokens'], embedding_type, mode, params)

    # Create document embeddings
    train_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in train_df['tokens']])
    val_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in val_df['tokens']])

    # Normalize features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    val_features = scaler.transform(val_features)

    # Train regressor
    regressor = HistGradientBoostingRegressor(
        learning_rate=params['learning_rate'],
        max_iter=params['max_iter'],
        max_depth=params['max_depth'],
        random_state=1234
    )
    regressor.fit(train_features, train_df['stars'])

    # Predict and evaluate
    val_pred = regressor.predict(val_features)
    return r2_score(val_df['stars'], val_pred)

# Settings
embedding_type, mode = 'fasttext', 'cbow'
n_trials = 30
n_jobs = 10
results = []

os.makedirs("figures", exist_ok=True)
os.makedirs("dataset", exist_ok=True)

print(f"\nOptimizing {embedding_type.upper()} - {mode.upper()}...")

study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=1234),
    study_name=f"{embedding_type}_{mode}"
)

study.optimize(
    partial(objective, embedding_type=embedding_type, mode=mode, train_df=train_df, val_df=val_df),
    n_trials=n_trials,
    n_jobs=n_jobs,
    show_progress_bar=True
)

best_params = study.best_trial.params
results.append({
    'setting': f"{embedding_type}-{mode}",
    'best_r2': study.best_value,
    **best_params
})

# Plot hyperparameter importance using matplotlib
optuna_viz.plot_param_importances(study)
plt.title(f"Hyperparameter Importance - {embedding_type}-{mode}")
plt.tight_layout()
plt.savefig(f"figures/param_importance_{embedding_type}_{mode}.png")
plt.close()

# Save results
results_df = pd.DataFrame(results)
print("\nHyperparameter Optimization Results for fastText-CBOW:")
print(results_df.to_string(index=False))

results_df.to_csv(f"dataset/hyperparameter_results_{embedding_type}_{mode}.csv", index=False)
print(f"\nResults saved to 'dataset/hyperparameter_results_{embedding_type}_{mode}.csv'")

c:\Users\PC\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


Section 3.2: Hyperparameter Tuning with Optuna for fastText-CBOW

Optimizing FASTTEXT - CBOW...


  0%|          | 0/28 [00:00<?, ?it/s]

: 

In [7]:
import pandas as pd
import numpy as np
import optuna
from functools import partial
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import word_tokenize
import string
import matplotlib.pyplot as plt
import optuna.visualization.matplotlib as optuna_viz
import os
import multiprocessing

# Set random seed for reproducibility
np.random.seed(1234)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)

print("Section 3.2: Hyperparameter Tuning with Optuna for doc2vec-DM")

# Load Data
train_df = pd.read_csv("dataset/train_reviews.csv")
val_df = pd.read_csv("dataset/val_reviews.csv")

def str_to_tokens(token_str):
    try:
        return eval(token_str) if isinstance(token_str, str) else token_str
    except:
        return []

# Convert tokens_str to lists
train_df['tokens'] = train_df['tokens_str'].apply(str_to_tokens)
val_df['tokens'] = val_df['tokens_str'].apply(str_to_tokens)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return word_tokenize(text)

# Apply preprocessing if tokens are missing
train_df['tokens'] = train_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)
val_df['tokens'] = val_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)

# Train embeddings
def train_embeddings(tokens_list, embedding_type, mode, params):
    vector_size = params['vector_size']
    window = params['window']
    epochs = params['epochs']
    min_count = params['min_count']

    if embedding_type == 'doc2vec':
        tagged_docs = [
            TaggedDocument(words=tokens, tags=[f"{row['user_id']}_{row['business_id']}"])
            for _, row in train_df.iterrows() for tokens in [row['tokens']]
        ]
        model = Doc2Vec(
            tagged_docs,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            dm=1 if mode == 'dm' else 0,
            epochs=epochs,
            seed=1234,
            alpha=params['alpha'],
            workers=1
        )
    else:
        raise ValueError(f"Invalid embedding type: {embedding_type}")
    
    return model

# Get document embedding
def get_doc_embedding(tokens, model, embedding_type, agg_type, handle_oov='ignore'):
    return model.infer_vector(tokens)

# Objective function for Optuna
def objective(trial, embedding_type, mode, train_df, val_df):
    params = {
        'vector_size': trial.suggest_int('vector_size', 50, 150, step=50),
        'window': trial.suggest_int('window', 3, 10),
        'epochs': trial.suggest_int('epochs', 5, 10, step=5),
        'min_count': trial.suggest_int('min_count', 1, 10),
        'alpha': trial.suggest_float('alpha', 0.01, 0.05, step=0.01),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_iter': trial.suggest_int('max_iter', 100, 300, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10)
    }
    params['agg_type'] = 'doc2vec'

    # Train embeddings
    model = train_embeddings(train_df['tokens'], embedding_type, mode, params)

    # Create document embeddings
    train_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in train_df['tokens']])
    val_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in val_df['tokens']])

    # Normalize features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    val_features = scaler.transform(val_features)

    # Train regressor
    regressor = HistGradientBoostingRegressor(
        learning_rate=params['learning_rate'],
        max_iter=params['max_iter'],
        max_depth=params['max_depth'],
        random_state=1234
    )
    regressor.fit(train_features, train_df['stars'])

    # Predict and evaluate
    val_pred = regressor.predict(val_features)
    return r2_score(val_df['stars'], val_pred)

# Settings
embedding_type, mode = 'doc2vec', 'dm'
n_trials = 50
n_jobs = 8
results = []

os.makedirs("figures", exist_ok=True)
os.makedirs("dataset", exist_ok=True)

print(f"\nOptimizing {embedding_type.upper()} - {mode.upper()}...")

study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=1234),
    study_name=f"{embedding_type}_{mode}"
)

study.optimize(
    partial(objective, embedding_type=embedding_type, mode=mode, train_df=train_df, val_df=val_df),
    n_trials=n_trials,
    n_jobs=n_jobs,
    show_progress_bar=True
)

best_params = study.best_trial.params
results.append({
    'setting': f"{embedding_type}-{mode}",
    'best_r2': study.best_value,
    **best_params
})

# Plot hyperparameter importance using matplotlib
optuna_viz.plot_param_importances(study)
plt.title(f"Hyperparameter Importance - {embedding_type}-{mode}")
plt.tight_layout()
plt.savefig(f"figures/param_importance_{embedding_type}_{mode}.png")
plt.close()

# Save results
results_df = pd.DataFrame(results)
print("\nHyperparameter Optimization Results for doc2vec-DM:")
print(results_df.to_string(index=False))

results_df.to_csv(f"dataset/hyperparameter_results_{embedding_type}_{mode}.csv", index=False)
print(f"\nResults saved to 'dataset/hyperparameter_results_{embedding_type}_{mode}.csv'")

Section 3.2: Hyperparameter Tuning with Optuna for doc2vec-DM

Optimizing DOC2VEC - DM...


Best trial: 18. Best value: 0.349287: 100%|██████████| 50/50 [2:10:02<00:00, 156.05s/it]
C:\Users\PC\AppData\Local\Temp\ipykernel_10024\593085586.py:154: ExperimentalWarning: plot_param_importances is experimental (supported from v2.2.0). The interface can change in the future.
  optuna_viz.plot_param_importances(study)



Hyperparameter Optimization Results for doc2vec-DM:
   setting  best_r2  vector_size  window  epochs  min_count  alpha  learning_rate  max_iter  max_depth
doc2vec-dm 0.349287          100       3      10          8   0.05        0.06995       200         10

Results saved to 'dataset/hyperparameter_results_doc2vec_dm.csv'


In [3]:
import pandas as pd
import numpy as np
import optuna
from functools import partial
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import word_tokenize
import string
import matplotlib.pyplot as plt
import optuna.visualization.matplotlib as optuna_viz
import os
import multiprocessing

# Set random seed for reproducibility
np.random.seed(1234)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)

print("Section 3.2: Hyperparameter Tuning with Optuna for doc2vec-DBOW")

# Load Data
train_df = pd.read_csv("dataset/train_reviews.csv")
val_df = pd.read_csv("dataset/val_reviews.csv")

def str_to_tokens(token_str):
    try:
        return eval(token_str) if isinstance(token_str, str) else token_str
    except:
        return []

# Convert tokens_str to lists
train_df['tokens'] = train_df['tokens_str'].apply(str_to_tokens)
val_df['tokens'] = val_df['tokens_str'].apply(str_to_tokens)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return word_tokenize(text)

# Apply preprocessing if tokens are missing
train_df['tokens'] = train_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)
val_df['tokens'] = val_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)

# Train embeddings
def train_embeddings(tokens_list, embedding_type, mode, params):
    vector_size = params['vector_size']
    window = params['window']
    epochs = params['epochs']
    min_count = params['min_count']

    if embedding_type == 'doc2vec':
        tagged_docs = [
            TaggedDocument(words=tokens, tags=[f"{row['user_id']}_{row['business_id']}"])
            for _, row in train_df.iterrows() for tokens in [row['tokens']]
        ]
        model = Doc2Vec(
            tagged_docs,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            dm=1 if mode == 'dm' else 0,
            epochs=epochs,
            seed=1234,
            alpha=params['alpha'],
            workers=1
        )
    else:
        raise ValueError(f"Invalid embedding type: {embedding_type}")
    
    return model

# Get document embedding
def get_doc_embedding(tokens, model, embedding_type, agg_type, handle_oov='ignore'):
    return model.infer_vector(tokens)

# Objective function for Optuna
def objective(trial, embedding_type, mode, train_df, val_df):
    params = {
        'vector_size': trial.suggest_int('vector_size', 50, 150, step=50),
        'window': trial.suggest_int('window', 3, 10),
        'epochs': trial.suggest_int('epochs', 5, 10, step=5),
        'min_count': trial.suggest_int('min_count', 1, 10),
        'alpha': trial.suggest_float('alpha', 0.01, 0.05, step=0.01),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_iter': trial.suggest_int('max_iter', 100, 300, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10)
    }
    params['agg_type'] = 'doc2vec'

    # Train embeddings
    model = train_embeddings(train_df['tokens'], embedding_type, mode, params)

    # Create document embeddings
    train_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in train_df['tokens']])
    val_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in val_df['tokens']])

    # Normalize features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    val_features = scaler.transform(val_features)

    # Train regressor
    regressor = HistGradientBoostingRegressor(
        learning_rate=params['learning_rate'],
        max_iter=params['max_iter'],
        max_depth=params['max_depth'],
        random_state=1234
    )
    regressor.fit(train_features, train_df['stars'])

    # Predict and evaluate
    val_pred = regressor.predict(val_features)
    return r2_score(val_df['stars'], val_pred)

# Settings
embedding_type, mode = 'doc2vec', 'dbow'
n_trials = 30
n_jobs = 8
results = []

os.makedirs("figures", exist_ok=True)
os.makedirs("dataset", exist_ok=True)

print(f"\nOptimizing {embedding_type.upper()} - {mode.upper()}...")

study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=1234),
    study_name=f"{embedding_type}_{mode}"
)

study.optimize(
    partial(objective, embedding_type=embedding_type, mode=mode, train_df=train_df, val_df=val_df),
    n_trials=n_trials,
    n_jobs=n_jobs,
    show_progress_bar=True
)

best_params = study.best_trial.params
results.append({
    'setting': f"{embedding_type}-{mode}",
    'best_r2': study.best_value,
    **best_params
})

# Plot hyperparameter importance using matplotlib
optuna_viz.plot_param_importances(study)
plt.title(f"Hyperparameter Importance - {embedding_type}-{mode}")
plt.tight_layout()
plt.savefig(f"figures/param_importance_{embedding_type}_{mode}.png")
plt.close()

# Save results
results_df = pd.DataFrame(results)
print("\nHyperparameter Optimization Results for doc2vec-DBOW:")
print(results_df.to_string(index=False))

results_df.to_csv(f"dataset/hyperparameter_results_{embedding_type}_{mode}.csv", index=False)
print(f"\nResults saved to 'dataset/hyperparameter_results_{embedding_type}_{mode}.csv'")

Section 3.2: Hyperparameter Tuning with Optuna for doc2vec-DBOW

Optimizing DOC2VEC - DBOW...


Best trial: 1. Best value: 0.424175: 100%|██████████| 30/30 [27:54<00:00, 55.81s/it] 
C:\Users\PC\AppData\Local\Temp\ipykernel_9944\2900203671.py:154: ExperimentalWarning: plot_param_importances is experimental (supported from v2.2.0). The interface can change in the future.
  optuna_viz.plot_param_importances(study)



Hyperparameter Optimization Results for doc2vec-DBOW:
     setting  best_r2  vector_size  window  epochs  min_count  alpha  learning_rate  max_iter  max_depth
doc2vec-dbow 0.424175           50       5      10          2   0.03       0.059124       200          6

Results saved to 'dataset/hyperparameter_results_doc2vec_dbow.csv'


In [9]:
import pandas as pd
import glob
import os

# Set output directory
output_dir = "dataset"
os.makedirs(output_dir, exist_ok=True)

# Find all hyperparameter result CSV files
result_files = glob.glob(os.path.join(output_dir, "hyperparameter_results_*.csv"))

# Check if any files were found
if not result_files:
    raise FileNotFoundError("No hyperparameter result CSV files found in 'dataset' directory. Please ensure the 6 codes for section 3.2 have been run.")

# Read and concatenate all CSV files
results_df = pd.concat([pd.read_csv(f) for f in result_files], ignore_index=True, sort=False)

# Round best_r2 to 2 decimal places
results_df['best_r2'] = results_df['best_r2'].round(2)

# Sort by setting for better readability (optional)
results_df = results_df.sort_values(by='setting')

# Save combined results to a single CSV file
output_path = os.path.join(output_dir, "hyperparameter_results.csv")
results_df.to_csv(output_path, index=False)

# Print combined results
print("\nCombined Hyperparameter Optimization Results:")
print(results_df.to_string(index=False))

print(f"\nCombined results saved to '{output_path}'")


Combined Hyperparameter Optimization Results:
      setting  best_r2  vector_size  window  epochs  min_count  alpha  learning_rate  max_iter  max_depth agg_type  min_n  max_n
 doc2vec-dbow     0.42           50       5      10          2   0.03       0.059124       200          6      NaN    NaN    NaN
   doc2vec-dm     0.35          100       3      10          8   0.05       0.069950       200         10      NaN    NaN    NaN
fasttext-cbow     0.30          150      10       5          4    NaN       0.171080       200          7  average    2.0    7.0
  fasttext-sg     0.43          100      10      10          6    NaN       0.083989       200          7  average    4.0    7.0
word2vec-cbow     0.42          150       8      10          3    NaN       0.088171       200          8  average    NaN    NaN
  word2vec-sg     0.43          150      10      10          5    NaN       0.032309       300          6  average    NaN    NaN

Combined results saved to 'dataset\hyperparameter

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import word_tokenize
from scipy.stats import spearmanr, kendalltau, pearsonr
from torchmetrics import MeanAbsolutePercentageError
import torch
import os
from sklearn.metrics import ndcg_score
import multiprocessing
import gc
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(1234)

# Create figures directory
os.makedirs("figures", exist_ok=True)

# Load data from Section 1
train_df = pd.read_csv("dataset/train_reviews.csv")
val_df = pd.read_csv("dataset/val_reviews.csv")
test_df = pd.read_csv("dataset/test_reviews.csv")

# Convert tokens_str to lists
def str_to_tokens(token_str):
    try:
        return eval(token_str) if isinstance(token_str, str) else token_str
    except:
        return []

train_df['tokens'] = train_df['tokens_str'].apply(str_to_tokens)
val_df['tokens'] = val_df['tokens_str'].apply(str_to_tokens)
test_df['tokens'] = test_df['tokens_str'].apply(str_to_tokens)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return word_tokenize(text)

# Apply preprocessing if tokens are missing
train_df['tokens'] = train_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)
val_df['tokens'] = val_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)
test_df['tokens'] = test_df.apply(lambda row: preprocess_text(row['text']) if not row['tokens'] else row['tokens'], axis=1)

# Load hyperparameter results from Section 3.2
results_df = pd.read_csv("dataset/hyperparameter_results.csv")

# Train embeddings
def train_embeddings(tokens_list, embedding_type, mode, params):
    vector_size = params['vector_size']
    window = params['window']
    epochs = params['epochs']
    min_count = params['min_count']

    if embedding_type == 'word2vec':
        model = Word2Vec(
            sentences=tokens_list,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            sg=1 if mode == 'sg' else 0,
            epochs=epochs,
            seed=1234,
            workers=multiprocessing.cpu_count()
        )
    elif embedding_type == 'fasttext':
        model = FastText(
            sentences=tokens_list,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            sg=1 if mode == 'sg' else 0,
            epochs=epochs,
            seed=1234,
            min_n=params['min_n'],
            max_n=params['max_n'],
            workers=multiprocessing.cpu_count()
        )
    elif embedding_type == 'doc2vec':
        tagged_docs = [
            TaggedDocument(words=tokens, tags=[f"{row['user_id']}_{row['business_id']}"])
            for _, row in train_df.iterrows() for tokens in [row['tokens']]
        ]
        model = Doc2Vec(
            tagged_docs,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            dm=1 if mode == 'dm' else 0,
            epochs=epochs,
            seed=1234,
            alpha=params.get('alpha', 0.025),
            min_alpha=params.get('alpha', 0.025) / 10,
            workers=multiprocessing.cpu_count()
        )
    else:
        raise ValueError(f"Invalid embedding type: {embedding_type}")
    
    return model

# Get document embedding
def get_doc_embedding(tokens, model, embedding_type, agg_type):
    if embedding_type == 'doc2vec':
        return model.infer_vector(tokens, epochs=50, alpha=0.025)  # Changed 'steps' to 'epochs'
    else:
        vectors = [model.wv[token] for token in tokens if token in model.wv]
        if not vectors:
            return np.zeros(model.vector_size)
        return np.mean(vectors, axis=0) if agg_type == 'average' else np.sum(vectors, axis=0)

# Compute CCC (Concordance Correlation Coefficient)
def compute_ccc(y_true, y_pred):
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    covar = np.cov(y_true, y_pred, bias=True)[0][1]
    ccc = (2 * covar) / (var_true + var_pred + (mean_true - mean_pred) ** 2)
    return ccc

# Compute NDCG@k per user
def compute_ndcg_per_user(y_true, y_pred, user_ids, k):
    ndcg_scores = []
    for user_id in np.unique(user_ids):
        user_mask = user_ids == user_id
        y_true_user = y_true[user_mask]
        y_pred_user = y_pred[user_mask]
        
        if len(y_true_user) >= k:
            sorted_indices = np.argsort(y_pred_user)[::-1]
            y_true_sorted = y_true_user[sorted_indices]
            y_pred_sorted = y_pred_user[sorted_indices]
            ndcg = ndcg_score([y_true_sorted], [y_pred_sorted], k=k)
            ndcg_scores.append(ndcg)
    
    return np.mean(ndcg_scores) if ndcg_scores else 0.0

# Evaluation function
def evaluate_model(setting, params, train_df, val_df, test_df):
    embedding_type, mode = setting.split('-')
    
    # Train embeddings
    model = train_embeddings(train_df['tokens'], embedding_type, mode, params)
    
    # Create document embeddings
    train_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in train_df['tokens']])
    val_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in val_df['tokens']])
    test_features = np.array([get_doc_embedding(tokens, model, embedding_type, params['agg_type']) for tokens in test_df['tokens']])
    
    # Normalize features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    val_features = scaler.transform(val_features)
    test_features = scaler.transform(test_features)
    
    # Train regressor
    regressor = HistGradientBoostingRegressor(
        learning_rate=params['learning_rate'],
        max_iter=params['max_iter'],
        max_depth=params['max_depth'],
        random_state=1234
    )
    regressor.fit(train_features, train_df['stars'])
    
    # Predict
    train_pred = regressor.predict(train_features)
    val_pred = regressor.predict(val_features)
    test_pred = regressor.predict(test_features)
    
    # Compute metrics
    def compute_metrics(y_true, y_pred, user_ids):
        metrics = {}
        metrics['R2'] = r2_score(y_true, y_pred)
        metrics['MAE'] = mean_absolute_error(y_true, y_pred)
        metrics['RMSE'] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics['MAPE'] = MeanAbsolutePercentageError()(torch.tensor(y_pred), torch.tensor(y_true)).item()
        metrics['CCC'] = compute_ccc(y_true, y_pred)
        metrics['Pearson'] = pearsonr(y_true, y_pred)[0]
        metrics['Spearman'] = spearmanr(y_true, y_pred)[0]
        metrics['Kendall'] = kendalltau(y_true, y_pred)[0]
        
        # Compute NDCG@k per user
        metrics['NDCG@10'] = compute_ndcg_per_user(y_true, y_pred, user_ids, k=10)
        metrics['NDCG@20'] = compute_ndcg_per_user(y_true, y_pred, user_ids, k=20)
        metrics['NDCG@50'] = compute_ndcg_per_user(y_true, y_pred, user_ids, k=50)
        
        return metrics
    
    results = {
        'setting': setting,
        'train': compute_metrics(train_df['stars'].values, train_pred, train_df['user_id'].values),
        'val': compute_metrics(val_df['stars'].values, val_pred, val_df['user_id'].values),
        'test': compute_metrics(test_df['stars'].values, test_pred, test_df['user_id'].values)
    }
    
    # Free memory
    del model, train_features, val_features, test_features, regressor
    gc.collect()
    
    return results

# Evaluate all settings
results = []
for _, row in results_df.iterrows():
    setting = row['setting']
    params = row.to_dict()
    del params['setting'], params['best_r2']  # Remove non-hyperparameter columns
    result = evaluate_model(setting, params, train_df, val_df, test_df)
    results.append(result)

# Format results for output
eval_results = []
for result in results:
    for split in ['train', 'val', 'test']:
        for metric, value in result[split].items():
            eval_results.append({
                'setting': result['setting'],
                'split': split,
                'metric': metric,
                'value': value
            })

# Save results
eval_results_df = pd.DataFrame(eval_results)
eval_results_df['value'] = eval_results_df['value'].round(2)  # Round all metrics to 2 decimal places
eval_results_df.to_csv("dataset/evaluation_results.csv", index=False)

# Print results
print("\nEvaluation Results:")
print(eval_results_df.pivot_table(index=['setting', 'split'], columns='metric', values='value').to_string())

# Visualization
# 1. Bar Plot for R²
r2_data = eval_results_df[eval_results_df['metric'] == 'R2']
plt.figure(figsize=(12, 6))
sns.barplot(x='setting', y='value', hue='split', data=r2_data)
plt.title('R² Scores Across Settings and Splits')
plt.xlabel('Setting')
plt.ylabel('R²')
plt.xticks(rotation=45)
plt.legend(title='Split')
plt.tight_layout()
plt.savefig('figures/r2_bar_plot.png')
plt.close()

# 2. Line Plot for NDCG@k
ndcg_data = eval_results_df[eval_results_df['metric'].isin(['NDCG@10', 'NDCG@20', 'NDCG@50'])]
plt.figure(figsize=(12, 6))
sns.lineplot(x='setting', y='value', hue='metric', style='split', markers=True, data=ndcg_data)
plt.title('NDCG@k Scores Across Settings and Splits')
plt.xlabel('Setting')
plt.ylabel('NDCG')
plt.xticks(rotation=45)
plt.legend(title='Metric / Split')
plt.tight_layout()
plt.savefig('figures/ndcg_line_plot.png')
plt.close()

print("\nResults saved to 'dataset/evaluation_results.csv'")
print("Plots saved to 'figures/r2_bar_plot.png' and 'figures/ndcg_line_plot.png'")


Evaluation Results:
metric                CCC  Kendall   MAE  MAPE  NDCG@10  NDCG@20  NDCG@50  Pearson    R2  RMSE  Spearman
setting       split                                                                                     
doc2vec-dbow  test   0.58     0.49  0.57  0.18     0.97     0.99     0.00     0.67  0.43  0.73      0.61
              train  0.72     0.62  0.48  0.15     0.98     0.98     0.98     0.79  0.60  0.60      0.75
              val    0.57     0.51  0.58  0.19     0.96     0.94     0.00     0.65  0.41  0.74      0.63
doc2vec-dm    test   0.54     0.45  0.61  0.19     0.97     0.98     0.00     0.63  0.37  0.76      0.56
              train  0.76     0.65  0.45  0.14     0.98     0.98     0.99     0.83  0.65  0.56      0.78
              val    0.53     0.47  0.61  0.20     0.96     0.89     0.00     0.60  0.36  0.78      0.59
fasttext-cbow test   0.48     0.42  0.64  0.20     0.97     0.99     0.00     0.58  0.31  0.80      0.53
              train  0.65     0.58

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
import multiprocessing

# Load validation data (ensure val_reviews.csv is in the dataset directory)
val_df = pd.read_csv("dataset/val_reviews.csv")

# Define get_doc_embedding function (assuming it's from previous code)
def get_doc_embedding(tokens, model, embedding_type, agg_type):
    if embedding_type == 'doc2vec':
        return model.infer_vector(tokens, epochs=50, alpha=0.025)
    else:
        vectors = [model.wv[token] for token in tokens if token in model.wv]
        if not vectors:
            return np.zeros(model.vector_size)
        return np.mean(vectors, axis=0) if agg_type == 'average' else np.sum(vectors, axis=0)

# Assume train_features and train_df are preloaded; load train data if needed
train_df = pd.read_csv("dataset/train_reviews.csv")
train_features = np.array([get_doc_embedding(tokens, Word2Vec(sentences=train_df['tokens'], vector_size=100, window=5, min_count=1, sg=1, seed=1234), 'word2vec', 'average') 
                         for tokens in train_df['tokens']])
train_features = StandardScaler().fit_transform(train_features)

# Predict top restaurants for user Tashalee using the best model (Word2Vec-SGNS)
tashalee_reviews = val_df[val_df['user_id'] == 'Tashalee']
if not tashalee_reviews.empty:
    # Load best parameters from Part 3.2 (assumed to be stored previously)
    best_params_word2vec_sg = {
        'vector_size': 100,  # Example; replace with actual value from hyperparameter_results.csv
        'window': 5,        # Example; replace with actual value
        'epochs': 10,       # Example; replace with actual value
        'min_count': 1,     # Example; replace with actual value
        'learning_rate': 0.1,  # Example; replace with actual value
        'max_iter': 200,    # Example; replace with actual value
        'max_depth': 5,     # Example; replace with actual value
        'agg_type': 'average'
    }

    # Train Word2Vec model with optimized parameters
    model = Word2Vec(
        sentences=train_df['tokens'],
        vector_size=best_params_word2vec_sg['vector_size'],
        window=best_params_word2vec_sg['window'],
        min_count=best_params_word2vec_sg['min_count'],
        sg=1,  # SGNS mode
        epochs=best_params_word2vec_sg['epochs'],
        seed=1234,
        workers=multiprocessing.cpu_count()
    )

    # Generate document embeddings for Tashalee's reviews
    features = np.array([get_doc_embedding(tokens, model, 'word2vec', best_params_word2vec_sg['agg_type']) 
                        for tokens in tashalee_reviews['tokens']])
    
    # Normalize the features using StandardScaler
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    # Train and predict with HistGradientBoostingRegressor
    from sklearn.ensemble import HistGradientBoostingRegressor
    regressor = HistGradientBoostingRegressor(
        learning_rate=best_params_word2vec_sg['learning_rate'],
        max_iter=best_params_word2vec_sg['max_iter'],
        max_depth=best_params_word2vec_sg['max_depth'],
        random_state=1234
    )
    # Assume train_features and train_df['stars'] are precomputed
    regressor.fit(train_features, train_df['stars'])
    preds = regressor.predict(features)

    # Sort restaurants by predicted score and select top 10
    top_restaurant_indices = np.argsort(preds)[::-1][:10]
    top_restaurants = tashalee_reviews.iloc[top_restaurant_indices]['business_id'].tolist()

    # Print and save the results
    print("\nTop 10 restaurants for Tashalee (Word2Vec-SGNS):", top_restaurants)
    with open('dataset/top_restaurants_tashalee.txt', 'w') as f:
        f.write("Top 10 restaurants for Tashalee (Word2Vec-SGNS):\n")
        for i, rest_id in enumerate(top_restaurants, 1):
            f.write(f"{i}. {rest_id}\n")
else:
    print("No reviews found for user Tashalee in validation set.")